# Initial setup

Install Bambu and required packages:

In [ ]:
!add-apt-repository -y ppa:git-core/ppa
!apt-get update
!apt-get install -y --no-install-recommends build-essential ca-certificates gcc-multilib git iverilog verilator
!wget https://release.bambuhls.eu:8080/appimage/bambu-showcase.AppImage
!chmod +x bambu-*.AppImage
!ln -sf $PWD/bambu-*.AppImage /bin/bambu
!ln -sf $PWD/bambu-*.AppImage /bin/spider
!ln -sf $PWD/bambu-*.AppImage /bin/tree-panda-gcc
!ln -sf $PWD/bambu-*.AppImage /bin/clang-12
!ln -sf $PWD/bambu-*.AppImage /bin/mlir-opt
!rm -rf PandA-bambu bambu-tutorial
!git clone --depth 1 --filter=blob:none --branch everest-school --sparse https://github.com/ferrandi/PandA-bambu.git
%cd PandA-bambu
!git sparse-checkout set documentation/everest_summer_school
%cd ..
!mv PandA-bambu/documentation/everest_summer_school/ bambu-tutorial

Check the installation:

In [ ]:
!bambu -h

In [ ]:
!mlir-opt --help

# Bambu inputs and outputs


Example C code in /content/bambu-tutorial/Exercise1/icrc.c

In [ ]:
%cd /content/bambu-tutorial/Exercise1
!bambu icrc.c --top-fname=icrc1 --simulator=VERILATOR --simulate --generate-tb=test_icrc1.xml -v2 --print-dot --pretty-print=a.c 2>&1 | tee icrc1.log

Inspect the generated files in the explorer tab on the left:

*   /content/bambu-tutorial/Exercise1/icrc1.v
*   /content/bambu-tutorial/Exercise1/simulate_icrc1.sh
*   /content/bambu-tutorial/Exercise1/synthesize_Synthesis_icrc1.sh
*   /content/bambu-tutorial/Exercise1/a.c



Visualize the FSM:

In [ ]:
from graphviz import Source
Source.from_file('HLS_output/dot/icrc1/HLS_STGraph.dot')

Try out:

* different target board
* different clock period
* VHDL instead of Verilog output
* different verbosity level

# HLS of an MLIR kernel

Source code: /content/bambu-tutorial/Exercise2/gemm_32.mlir and /content/bambu-tutorial/Exercise2/helmholtz.mlir

In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise2/
!mlir-opt gemm_32.mlir -lower-affine -convert-scf-to-cf -convert-arith-to-llvm -convert-memref-to-llvm -convert-math-to-llvm -convert-func-to-llvm='use-bare-ptr-memref-call-conv=1' -reconcile-unrealized-casts -o gemm_32.llvm.mlir
!mlir-translate gemm_32.llvm.mlir -opaque-pointers=0 --mlir-to-llvmir -o gemm_32.ll
!bambu gemm_32.ll --simulate --generate-tb=gemm_32_test.xml --no-clean --compiler=I386_CLANG12 --top-fname=gemm_32 --simulator=VERILATOR -v4 --disable-function-proxy --print-dot |& tee log.txt

Try out:
* synthesize the Helmholtz kernel
* use the --generate-interface option
* apply mlir-opt optimizations before synthesis
* disable function proxies to allocate floating point units in parallel

# OpenMP parallel for

Source code: /content/bambu-tutorial/Exercise3/trinityq4/lubm_trinityq4.c


In [ ]:
%cd /content/bambu-tutorial/Exercise3/
!bambu trinityq4/lubm_trinityq4.c --top-fname=search common/atominIncrement.c common/data.c -Icommon/ --compiler=I386_GCC49 --experimental-set=BAMBU -O3 --std=c99 -fno-delete-null-pointer-checks --channels-type=MEM_ACC_11 --memory-allocation-policy=NO_BRAM --device-name=xc7vx690t-3ffg1930-VVD --clock-period=10 -DMAX_VERTEX_NUMBER=26455 -DMAX_EDGE_NUMBER=100573 -DN_THREADS=1 --mem-delay-read=20 --mem-delay-write=20 --generate-tb=$root_dir/test-1.xml --simulator=VERILATOR --simulate -v3 --pragma-parse --num-accelerators=2 --memory-banks-number=4 --channels-number=2 --context_switch=4 |& tee log.txt

# ap_types and ac_types support

Synthesis of an acceleratore with input and output fifo ports.

In [ ]:
%cd /content/bambu-tutorial/Exercise4
!bambu ap_example.cpp --simulate --no-clean --compiler=I386_CLANG12 --generate-interface=INFER --top-fname=gcd --simulator=VERILATOR

## Custom floating point synthesis



In [ ]:
%cp /content/bambu-tutorial/Exercise2/gemm_32.ll /content/bambu-tutorial/Exercise4
!bambu gemm_32.ll --simulate --no-clean --compiler=I386_CLANG12 --top-fname=gemm_32 --simulator=VERILATOR -v4 --disable-function-proxy --print-dot --fp-format="gemm_32*8*7*-127*0*0*1*0*U" --max-ulp=2000000 |& tee log.bfloat16.txt